# 2025 March Madness Tournament Winner Prediction

In [1]:
import pandas as pd

In [2]:
historical_seasons = pd.read_csv('cbb.csv')
historical_seasons.head()

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
0,North Carolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,...,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016
1,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,...,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1.0,2015
2,Michigan,B10,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,...,30.0,54.7,46.8,35.2,33.2,65.9,6.9,2ND,3.0,2018
3,Texas Tech,B12,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,...,36.6,52.8,41.9,36.5,29.7,67.5,7.0,2ND,3.0,2019
4,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,...,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1.0,2017


In [3]:
current_season_basic = pd.read_csv('2024-25 College Basketball Basic Stats.csv')
current_season_basic.head()

,Rk,School,G,W▼,L,W-L%,SRS,SOS,Unnamed: 8,W,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Auburn,27,25,2,0.926,30.31,14.13,NaN,13,...,406,548,0.741,333,1044,449,193,176,246,490
1,2,Drake,28,25,3,0.893,9.20,0.47,NaN,15,...,422,600,0.703,285,863,374,250,62,324,450
2,3,Duke,28,25,3,0.893,30.56,8.99,NaN,16,...,405,520,0.779,306,1073,482,195,98,267,445
3,4,High Point,30,25,5,0.833,5.42,-5.29,NaN,13,...,514,666,0.772,329,1082,391,183,86,309,553
4,5,Saint Mary's (CA),29,25,4,0.862,18.50,5.54,NaN,15,...,369,532,0.694,403,1143,464,192,117,282,434


In [4]:
current_season_basic = current_season_basic.drop(columns=['Unnamed: 8', 'Rk'])

In [10]:
current_season_basic.columns = ['School', 'Games', 'W', 'L', 'W-L%', 'SRS', 'SOS', 'Cof. W', 'Cof. L', 'Home W', 'Home L',
                                                     'Away W', 'Away L', 'Team Pts', 'Opponent Pts', 'MP', 'FG', 'FGA',	'FG%', '3P', '3PA',	'3P%', 'FT', 'FTA',	'FT%',
                                                    'ORB','TRB', 'AST',	'STL',	'BLK',	'TOV',	'PF']
urrent_season_basic.head()

ValueError: Length mismatch: Expected axis has 36 elements, new values have 32 elements

In [13]:
current_season_adv = pd.read_csv('2024-25 College Basketball Advanced Stats.csv')
current_season_adv.head()

,Rk,School,G,W▼,L,W-L%,SRS,SOS,Unnamed: 8,W,...,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
0,1,Auburn,27,25,2,0.926,30.31,14.13,NaN,13,...,0.407,0.584,53.8,55.4,10.3,17.1,0.554,11.2,35.7,0.240
1,2,Drake,28,25,3,0.893,9.20,0.47,NaN,15,...,0.368,0.589,55.7,54.3,14.5,7.7,0.558,16.1,36.5,0.300
2,3,Duke,28,25,3,0.893,30.56,8.99,NaN,16,...,0.453,0.612,55.9,59.7,10.5,9.9,0.580,12.4,34.7,0.247
3,4,High Point,30,25,5,0.833,5.42,-5.29,NaN,13,...,0.392,0.603,54.1,45.9,9.1,7.9,0.565,13.1,35.0,0.297
4,5,Saint Mary's (CA),29,25,4,0.862,18.50,5.54,NaN,15,...,0.341,0.544,57.0,57.9,10.2,10.7,0.517,12.4,39.7,0.212


In [14]:
current_season_adv = current_season_adv.drop(columns=['Unnamed: 8'])

In [ ]:
merged_df = 